In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# --- PASSO 1: Carregar os Dados e Criar o Perfil do Cliente ---

# Carregue seus dados (do CSV ou do DB)
df = pd.read_csv('viagens_clickbus.csv')
df['date_purchase'] = pd.to_datetime(df['date_purchase'])

# Agrupar por cliente para entender o comportamento (código que já tínhamos)
perfil_cliente = df.groupby('fk_contact').agg(
    total_gasto=('gmv_success', 'sum'),
    total_viagens=('nk_ota_localizer_id', 'count'),
    destinos_unicos=('place_destination_departure', 'nunique'),
    primeira_compra=('date_purchase', 'min'),
    ultima_compra=('date_purchase', 'max')
).reset_index()

# --- PASSO 2: Engenharia de Features e Criação do Alvo (Target) ---

# O objetivo é prever a recompra em 30 dias.
# Como não temos dados futuros para saber quem realmente comprou de novo,
# vamos SIMULAR essa variável alvo de forma realista para demonstrar o modelo.
# A simulação assume que clientes com mais gastos têm mais chance de recomprar.

# Normalizar o gasto total entre 0 e 1 para usar como probabilidade
prob_recompra = perfil_cliente['total_gasto'] / perfil_cliente['total_gasto'].max()

# Criar o alvo 'recompra_em_30d' (1 para SIM, 0 para NÃO)
# Usamos uma distribuição binomial com base na probabilidade calculada
perfil_cliente['recompra_em_30d'] = np.random.binomial(1, p=prob_recompra, size=len(perfil_cliente))


# --- PASSO 3: Preparar os Dados para o Modelo ---

# X são as features que usamos para fazer a previsão
X = perfil_cliente[['total_gasto', 'total_viagens', 'destinos_unicos']]

# y é o alvo que queremos prever
y = perfil_cliente['recompra_em_30d']

# Dividir os dados em conjunto de treino e conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# --- PASSO 4: Treinar e Avaliar o Modelo de Classificação Binária ---

# Usaremos o RandomForestClassifier
model_recompra = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Treinar o modelo
model_recompra.fit(X_train, y_train)
print("Modelo de previsão de recompra (Slide 2) treinado com sucesso!")

# Fazer previsões no conjunto de teste
y_pred = model_recompra.predict(X_test)

# Calcular e exibir a acurácia e outras métricas
accuracy = accuracy_score(y_test, y_pred)

print("\n--- AVALIAÇÃO DO MODELO (SLIDE 2 - O TIMING É TUDO) ---")
print(f"Objetivo: Prever se um cliente fará uma recompra nos próximos 30 dias. ")
print(f"Acurácia do Modelo: {accuracy:.2%}")
print("\nRelatório de Classificação Detalhado:")
print(classification_report(y_test, y_pred, target_names=['Não Recomprou', 'Recomprou']))

ModuleNotFoundError: No module named 'sklearn'